In [83]:
import pandas as pd
import numpy  as np

import matplotlib.pyplot as plt

import torch

import torch.nn as nn

In [ ]:
data = pd.read_csv('train.csv')

data = data[:100]

data

In [ ]:
data['Log1pSalary'] = np.log1p(data['SalaryNormalized']).astype('float32')

plt.figure(figsize=[8, 4])
plt.subplot(1, 2, 1)
plt.hist(data["SalaryNormalized"], bins=20);

plt.subplot(1, 2, 2)
plt.hist(data['Log1pSalary'], bins=20);

In [ ]:
text_columns = ["Title", "FullDescription"]

categorical_columns = ["Category", "Company", "LocationNormalized", "ContractType", "ContractTime"]

TARGET_COLUMN = "Log1pSalary"

data[categorical_columns] = data[categorical_columns].fillna('NaN')

data.head(3)

In [ ]:
import nltk

from nltk import WordPunctTokenizer

tokenizer = nltk.tokenize.WordPunctTokenizer()

# Will have Title as string of space separated tokens 
data['Title'] = data['Title'].apply(lambda x: " ".join((tokenizer.tokenize(str(x).lower()))))
# Will have FullDescription as string of space separated tokens 
data['FullDescription'] = data['FullDescription'].apply(lambda x: " ".join((tokenizer.tokenize(str(x).lower()))))

In [ ]:
from collections import Counter

In [ ]:
title_tokens_count = Counter(sum([i.split() for i in data['Title']], []))
descr_tokens_count = Counter(sum([i.split() for i in data['FullDescription']], []))

tokens_count       = title_tokens_count + descr_tokens_count

In [ ]:
print(f'{len(tokens_count)} unique tokens')

In [ ]:
min_count = 10
tokens    = [token for token, count in tokens_count.items() if count > min_count]
# Add special tokens for unknown and padding
UNK, PAD  = "UNK", "PAD"
tokens    = [UNK, PAD] + tokens

tokens[:10]

In [ ]:
print("Vocabulary size is", len(tokens))

In [ ]:
# Toke index
token_to_idx = {token: tokens.index(token) for token in tokens}

In [ ]:
UNK_IX, PAD_IX = map(token_to_idx.get, [UNK, PAD])


def as_matrix(sequences):
    sequences = list(map(tokenizer.tokenize, sequences))
    
    max_len = max(map(len, sequences))
    
    matrix = np.full((len(sequences), max_len), np.int32(PAD_IX))
    
    for i, seq in enumerate(sequences):
        row_idxs = [token_to_idx.get(token, UNK_IX) for token in seq]
        matrix[i, :len(row_idxs)] = row_idxs
    
    return matrix

In [ ]:
as_matrix(['Hi, I am cat',
           'I like data science'])

In [ ]:
k = 10

top_k_companies_names, top_k_companies_counts = zip(*Counter(data['Company']).most_common(k))

unique_top_k_companies_names = set(top_k_companies_names)

data['Company'] = data['Company'].apply(lambda comp: comp if comp in unique_top_k_companies_names else "Other")

data['Company']

In [ ]:
from sklearn.feature_extraction import DictVectorizer

categorical_vectorizer = DictVectorizer(dtype=np.float32, sparse=False)

categorical_vectorizer.fit(data[categorical_columns].apply(dict, axis=1))

In [ ]:
from sklearn.model_selection import train_test_split

data_train, data_val = train_test_split(data, test_size=0.2, random_state=42)
data_train.index     = range(len(data_train))
data_val.index       = range(len(data_val))

print("Train size = ", len(data_train))
print("Validation size = ", len(data_val))

In [ ]:
def make_batch(data, batch_size, title_word_dropout=0, descr_word_dropout=0):
    batch = {}
    
    batch["FullDescription"] = as_matrix(data["FullDescription"].values)
    batch["Title"] = as_matrix(data["Title"].values)
    
    batch['Categorical'] = categorical_vectorizer.transform(data[categorical_columns].apply(dict, axis=1))
    
    if title_word_dropout != 0:
        batch["Title"] = apply_word_dropout(batch["Title"], keep_prop= 1. - title_word_dropout)
    if descr_word_dropout !=0:
        batch["FullDescription"] = apply_word_dropout(batch["FullDescription"], keep_prop= 1. - descr_word_dropout)
    if TARGET_COLUMN in data.columns:
        batch[TARGET_COLUMN] = data[TARGET_COLUMN].values

    
    return to_tensors(batch)



def apply_word_dropout(matrix, keep_prop, replace_with=UNK_IX, pad_ix=PAD_IX,):
    """
    matrix: numpy array, needed to use matrix != pad_ix mask condition
    because we dont want to set UNK to the PAD
    """
    dropout_mask = np.random.choice(2, np.shape(matrix), p=[keep_prop, 1 - keep_prop])
    dropout_mask &= matrix != pad_ix
    return np.choose(dropout_mask, [matrix, np.full_like(matrix, replace_with)])



def to_tensors(batch):
    batch_tensors = dict()
    for key, arr in batch.items():
        batch_tensors[key] = torch.tensor(arr)
    return batch_tensors

In [ ]:
class Model(nn.Module):
    
    def __init__(self, embedding_dim=64, n_tokens=len(tokens), n_cat_features=len(categorical_vectorizer.vocabulary_)):
        super().__init__()
        
        self.full_description_emb = nn.Embedding(n_tokens, embedding_dim)
        self.title_emb = nn.Embedding(n_tokens, embedding_dim)
        
        self.full_description_seq = nn.Sequential(
            nn.Conv1d(in_channels=embedding_dim, out_channels=16, kernel_size=3, stride=1),
            nn.Tanh()
        )
        
        self.title_seq = nn.Sequential(
            nn.Conv1d(in_channels=embedding_dim, out_channels=16, kernel_size=3, stride=1),
            nn.Tanh()
        )
        
        self.categorical_seq = nn.Linear(in_features=n_cat_features, out_features=16)
        
        self.final = nn.Linear(in_features=16 * 3, out_features=1)
    
    def forward(self, batch):
        full_description = batch["FullDescription"]
        title = batch["Title"]
        categorical = batch["Categorical"]
        
        full_description_emb = self.full_description_emb(full_description).transpose(1, 2)
        full_description_output = self.full_description_seq(full_description_emb).mean(dim=2)
        
        title_emb = self.title_emb(title).transpose(1, 2)
        title_output = self.title_seq(title_emb).mean(dim=2)
        
        categorical_output = self.categorical_seq(categorical)

        merged_output = torch.cat([full_description_output, title_output, categorical_output], dim=1)

        output = self.final(merged_output).reshape(-1)
        return output

In [ ]:
def iterate_minibatches(data, batch_size=256, shuffle=True, device=torch.device('cpu'), **kwargs):
    indices = np.arange(len(data))
    if shuffle:
        indices = np.random.permutation(indices)

    for start in range(0, len(indices), batch_size):
        batch = make_batch(data.iloc[indices[start : start + batch_size]], batch_size=batch_size, **kwargs)
        yield batch

In [ ]:
import tqdm

BATCH_SIZE = 16
EPOCHS = 20
DEVICE = torch.device('cpu')


def print_metrics(model, data, batch_size=BATCH_SIZE, name="", **kw):
    squared_error = abs_error = num_samples = 0.0
    model.eval()
    with torch.no_grad():
        for batch in iterate_minibatches(data, batch_size=batch_size, shuffle=False, **kw):
            batch_pred = model(batch)
            squared_error += torch.sum(torch.square(batch_pred - batch[TARGET_COLUMN]))
            abs_error += torch.sum(torch.abs(batch_pred - batch[TARGET_COLUMN]))
            num_samples += len(batch_pred)
    mse = squared_error.detach().cpu().numpy() / num_samples
    mae = abs_error.detach().cpu().numpy() / num_samples
    print("%s results:" % (name or ""))
    print("Mean square error: %.5f" % mse)
    print("Mean absolute error: %.5f" % mae)
    return mse, mae

In [ ]:
model     = Model()
criterion = nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

In [ ]:
for epoch in range(EPOCHS):
    print(f"epoch: {epoch}")
    model.train()
    for i, batch in tqdm.notebook.tqdm(enumerate(
            iterate_minibatches(data_train, batch_size=BATCH_SIZE, device=DEVICE)),
            total=len(data_train) // BATCH_SIZE
        ):
        pred = model(batch)
        loss = criterion(pred, batch[TARGET_COLUMN])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if epoch % 10 == 0: 
        print_metrics(model, data_val)

In [ ]:
def explain(model, sample, col_name, sensitivity=1):
    sample_col_tokens = [tokens[token_to_idx.get(tok, 0)] for tok in sample[col_name].split()]

    data_drop_one_token = pd.DataFrame([sample] * (len(sample_col_tokens)))
    data_full           = pd.DataFrame([sample] * (len(sample_col_tokens)))

    # Since we want to try to drop each word we will have
    # to set 'number of words' times UNC to different position
    batch_size = len(data_full)

    text_col_idx = data_full.columns.get_loc(col_name)
    text = data_drop_one_token.iloc[0, text_col_idx].split()

    for i in range(batch_size):
        data_drop_one_token.iloc[i, text_col_idx] = ' '.join([UNK if j == i else t for j, t in enumerate(text)])

    data_drop_one_token_batch  = make_batch(data_drop_one_token, batch_size)
    data_drop_one_token_output = model.forward(data_drop_one_token_batch)


    data_baseline_batch = make_batch(data_full, batch_size)
    data_baseline = model.forward(data_baseline_batch)

    # if diff < 0 it means that without word, the price is higher,
    # so probably its red flag, lets make it red
    # and if diff > 0 it means that without price is lower
    diff = (data_baseline - data_drop_one_token_output).detach() * sensitivity
    return list(zip(text, diff))
    
# explain(model=model, sample=data.iloc[2], col_name='Title')

In [ ]:
from IPython.display import HTML, display_html


def draw_html(tokens_and_weights, cmap=plt.get_cmap("bwr"), display=True,
              token_template="""<span style="background-color: {color_hex}">{token}</span>""",
              font_style="font-size:14px;"):
    
    def get_color_hex(weight):
        # weight here is out diff so when its < 0 then it means without this word
        # price would be higher so its reg flag to delete
        # Sigmoid kind of
        rgba = cmap(1. / (1 + np.exp(weight.detach().numpy())), bytes=True)
        return '#%02X%02X%02X' % rgba[:3]
    
    tokens_html = [
        token_template.format(token=token, color_hex=get_color_hex(weight))
        for token, weight in tokens_and_weights
    ]
    
    raw_html = """<p style="{}">{}</p>""".format(font_style, ' '.join(tokens_html))
    if display:
        display_html(HTML(raw_html))
        
    return raw_html

In [ ]:
explain_title = explain(model=model, sensitivity=1, sample=data.iloc[23], col_name='Title')
draw_html([(text, diff) for text, diff in explain_title]);

explain_descr = explain(model=model, sensitivity=10, sample=data.iloc[23], col_name='FullDescription')
draw_html([(text, diff) for text, diff in explain_descr]);